In [2]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import nest_asyncio
import wandb

# Project imports
import stan
import httpstan

In [ ]:
# Weights & biases
# NOTE: Only turn this on when you think you have got everything working
wandb_run = False
if wandb_run:
    wandb.init(project='HW1-Q6', entity='cs532-2022')

Bayesian neural network
```
(let [weight-prior (normal 0 1)
      W_0 (foreach 10 []
            (foreach 1 [] (sample weight-prior)))
      W_1 (foreach 10 []
            (foreach 10 [] (sample weight-prior)))
      W_2 (foreach 1 []
            (foreach 10 [] (sample weight-prior)))

      b_0 (foreach 10 []
            (foreach 1 [] (sample weight-prior)))
      b_1 (foreach 10 []
            (foreach 1 [] (sample weight-prior)))
      b_2 (foreach 1 []
            (foreach 1 [] (sample weight-prior)))

      x   (mat-transpose [[1] [2] [3] [4] [5]])
      y   [[1] [4] [9] [16] [25]]
      h_0 (mat-tanh (mat-add (mat-mul W_0 x)
                             (mat-repmat b_0 1 5)))
      h_1 (mat-tanh (mat-add (mat-mul W_1 h_0)
                             (mat-repmat b_1 1 5)))
      mu  (mat-transpose
            (mat-tanh (mat-add (mat-mul W_2 h_1)
                               (mat-repmat b_2 1 5))))]
(foreach 5 [y_r y
            mu_r mu]
   (foreach 1 [y_rc y_r
               mu_rc mu_r]
      (observe (normal mu_rc 1) y_rc)))
[W_0 b_0 W_1 b_1])
```

In [ ]:
# Requied for stan in jupyter
nest_asyncio.apply()

# Environment variables for Stan compilation
%env CC=gcc
%env CXX=gcc

# Stan info
print('Stan version:', stan.__version__)
print('Stan location:', stan.__file__)

In [ ]:
# Read Stan code
code_file = 'Q6.stan' # NOTE: You will need to write the model in Q6.stan
with open(code_file) as f:
    stan_code = f.read()
f.close()
print('Stan code:\n', stan_code)

In [ ]:
# Parameters

# Observations
# NOTE: Fill this in!
data = None

# Options
force_rebuild_model = False
num_chains = 4
num_samples = int(1e4)

In [ ]:
# Get the the name of the folder where your model is saved then delete
if force_rebuild_model:
    model_name = httpstan.models.calculate_model_name(stan_code)
    print('Stan model name:', model_name)
    httpstan.cache.delete_model_directory(model_name)

In [17]:
# Run Stan
posterior = stan.build(stan_code, data=data, random_seed=1)
fit = posterior.sample(num_chains=num_chains, num_samples=num_samples)

# W&B logging
if wandb_run:
    for W0, W1, W2, b0, b1, b2 in zip(fit['W0'], fit['W1'], fit['W2'], fit['b0'], fit['b1'], fit['b2']):
        wandb.log({'W0': W0, 'W1': W1, 'b0': b0, 'b1': b1, 'W2': W2, 'b2': b2})

In [ ]:
# Print information about outputs
for array in ['W0', 'W1', 'W2', 'b0', 'b1', 'b2']:
    print(array, type(fit[array]), fit[array].shape)

In [ ]:
# Plot posterior distributions
plt.subplots(2, 3, figsize=(12, 8))

# W0
plt.subplot(2, 3, 1)
for i in range(10):
    plt.hist(fit['W0'][i, 0], density=True, bins='auto', alpha=0.3)
plt.title(r'$W_0$')
plt.yticks([])

# W1
plt.subplot(2, 3, 2)
for i in range(10):
    for j in range(10):
        plt.hist(fit['W1'][i, j], density=True, bins='auto', alpha=0.1)
plt.title(r'$W_1$')
plt.yticks([])

# W2
plt.subplot(2, 3, 3)
for i in range(10):
    plt.hist(fit['W2'][0, i], density=True, bins='auto', alpha=0.1)
plt.title(r'$W_2$')
plt.yticks([])

# b0
plt.subplot(2, 3, 4)
for i in range(10):
    plt.hist(fit['b0'][i, 0], density=True, bins='auto', alpha=0.3)
plt.title(r'$b_0$')
plt.yticks([])

# b1
plt.subplot(2, 3, 5)
for i in range(10):
    plt.hist(fit['b1'][i, 0], density=True, bins='auto', alpha=0.3)
plt.title(r'$b_1$')
plt.yticks([])

# b2
plt.subplot(2, 3, 6)
plt.hist(fit['b2'][0, 0], density=True, bins='auto', alpha=0.1)
plt.title(r'$b_2$')
plt.yticks([])

if wandb_run: wandb.log({'Question 6; posterior': wandb.Image(plt)})

plt.show()

In [ ]:
# Posterior predictive
plt.hist(fit['y_tilde'][0], density=True, bins='auto')
plt.xlabel('Posterior predictive at new datum: %1.1f'%(data['x_new']))
plt.yticks([])
if wandb_run: wandb.log({'Question 6; posterior predictive': wandb.Image(plt)})
plt.show()

In [ ]:
# Neural network in numpy
def model(W0, b0, W1, b1, W2, b2, x):
    # NOTE: Fill this in!
    y = np.tanh(W2*np.tanh(W1*(np.tanh(x*W0+b0))+b1)+b2)
    return None

In [ ]:
# Parameters
xmin = 0.; xmax = 6.; nx = 33
x = np.array([np.linspace(xmin, xmax, nx)])
n = int(1e3)

# Draw samples to form the posterior predictive
ys = []
for i in range(n):
    W0 = fit['W0'][:, :, i]
    W1 = fit['W1'][:, :, i]
    W2 = fit['W2'][:, :, i]
    b0 = fit['b0'][:, i]
    b1 = fit['b1'][:, i]
    b2 = fit['b2'][:, i]
    y = model(W0, b0, W1, b1, W2, b2, x)
    if i == 0:
        print('Example y:', y, y.shape, '\n')
    ys.append(y) # Output will also be a matrix
ys = np.array(ys) # Convert list to numpy array

In [ ]:
# Plot posterior predictive distribution
plt.fill_between(x[0], np.percentile(ys, 2.28, axis=0)[0], np.percentile(ys, 97.72, axis=0)[0], color='C0', alpha=0.25) # 2-sigma
plt.fill_between(x[0], np.percentile(ys, 15.87, axis=0)[0], np.percentile(ys, 84.13, axis=0)[0], color='C0', alpha=0.5) # 1-sigma
plt.plot(x[0], ys.mean(axis=0)[0], color='C0', label='Model predictions')
plt.scatter(data['x'], data['y'], marker='o', color='black', label='Training data')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
if wandb_run: wandb.log({'Question 6; predictive': wandb.Image(plt)})
plt.show()

In [ ]:
# Finalize W&B
if wandb_run:
    wandb.finish()